## Run simulation and optimizations for the folded cascode example


In [1]:
import json
import circuits as cir
import optimizers as opt
import ngspice
import pandas as pd

import importlib
importlib.reload(cir)
importlib.reload(opt)
importlib.reload(ngspice)


<module 'ngspice' from '/home/nlourenco/Work/workspace-haicas/BAIcO/src/ngspice.py'>

In [2]:
#load the cicruit definitions and targets defined in circuit_setup.json
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results Typ Only")

df = pd.DataFrame(simulation_result[0]).transpose()

df['CL'] *= 1e12  
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)


Simulation Results Typ Only
    CL[pF]  DELTA_MNM1  DELTA_MNM2  DELTA_MNM3  DELTA_MNM4  DELTA_MNM5  \
TT     6.0    0.585495    0.739938    0.617564    0.619046    0.413995   

    DELTA_MNM6  DELTA_MNM7  DELTA_MNM8  DELTA_MPM0  ...  VOV_MNM5  VOV_MNM6  \
TT    0.639173    0.402403    0.402872    0.472546  ...  0.001432  0.000963   

    VOV_MNM7  VOV_MNM8  VOV_MPM0  VOV_MPM1  VOV_MPM3  VOV_MPM4  VOV_MPM5  \
TT -0.016224 -0.016224   0.37476  0.103984  1.307041  1.307041  0.405695   

    VOV_MPM6  
TT  0.407171  

[1 rows x 42 columns]


In [3]:
#load the cicruit definitions and targets defined in circuit_setup.json with corners from corners.inc
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/")

#load a sample sizing
with open("./circuit_examples/ptm130_folded_cascode/sizing_example.json", 'r') as file:
    sizing = json.load(file)

simulation_result = folded_cascode.simulate(sizing)


print("Simulation Results All Corners")
df = pd.DataFrame(simulation_result[0]).transpose()

# rescale CL to show not 0 in df
df['CL'] *= 1e12
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)

Simulation Results All Corners
                       CL[pF]       GDC        GPS  INOISE_TOTAL  \
SS_VDD_MIN_TEMP_MAX       6.0 -86.95084 -75.768650  1.204242e-01   
SS_VDD_MAX_TEMP_MIN       6.0  69.53510 -14.417560  4.667157e-07   
FF_VDD_MAX_TEMP_MAX       6.0  87.51435 -18.703350  7.395473e-07   
FF_VDD_MAX_TEMP_MIN       6.0  68.64928 -14.566390  2.497138e-07   
TT                        6.0  88.39221  -3.418371  6.702275e-07   
FF_VDD_MIN_TEMP_MAX       6.0  98.37739   6.757086  3.649574e-07   
FF_VDD_MIN_TEMP_MIN       6.0  72.23919 -21.388510  5.345136e-07   
SS_VDD_MIN_TEMP_MIN       6.0  80.83673  -5.460861  4.928744e-07   
SNFP_VDD_MIN_TEMP_MAX     6.0 -87.04048 -63.453340  1.321339e-01   
SNFP_VDD_MAX_TEMP_MAX     6.0  85.08479  -7.335310  8.044332e-07   
SNFP_VDD_MAX_TEMP_MIN     6.0  63.25632 -13.049850  3.560017e-07   
FNSP_VDD_MAX_TEMP_MIN     6.0  75.23211 -16.525670  3.501699e-07   
SNFP_VDD_MIN_TEMP_MIN     6.0  68.89267 -18.217450  5.405864e-07   
FNSP_VDD_MAX_TEMP

In [4]:
obj,cstr,log = folded_cascode.target.evaluate(simulation_result[0])

print("Objectives")

print( {folded_cascode.objectives[i][0]:(obj[i]*folded_cascode.objectives[i][1]) for i in range(len(obj))})

Objectives
{'IDD': 0.001285138, 'GBW': -inf, 'GDC': -98.38497}


## Optimizing in Nominal conditions

Outputs are save to the output_folder

This can take a while.


In [5]:
import random
from datetime import datetime
import numpy as np
import pandas as pd
import os


seed = 42
np.random.seed(seed)
random.seed(seed)
nsga2 = opt.NSGA2()

output_folder = "./run/ptm130_folded_cascode/"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


i = 0
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)
for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        folded_cascode,pop_size=256, evaluations=256*100, mutation=0.1, crossover=0.6):
        
        print(i, pop_cstr[pop_cstr.argmax()], datetime.now().time())
        with open("{}history256_{}_{}.json".format(output_folder,seed, i), "w") as file: 
            json.dump(
            {
                "pop":json.loads(pd.DataFrame(data=pop, columns=folded_cascode.parameters).to_json(orient="records")),
                "obj":json.loads(pd.DataFrame(data=pop_obj, columns=folded_cascode.objectives).to_json(orient="records")),
                "cstr":pop_cstr.tolist(),
                "data":pop_data,
                "evals":evals,
                "fn":[ str(fn) for fn in front_no.tolist()]
            },
            file  
            )
        i = i + 1


0 [-2.46806649] 23:23:14.820405
1 [-1.08976006] 23:23:24.402123
2 [-1.08976006] 23:24:13.973775
3 [-1.08976006] 23:25:24.630437
4 [-1.08976006] 23:25:38.176217
5 [-0.42640318] 23:26:10.314253
6 [-0.42640318] 23:26:18.981350


KeyboardInterrupt: 

In [ ]:
import plot_utils 
importlib.reload(plot_utils)

plot_utils.plt_pof(["{}history256_{}_{}.json".format(output_folder,seed, 100)], scales = [("[uA]", 1e6),("[MHz]", 1e-6),("[dB]", 1.0)])



In [ ]:
import sys
print(sys.path)



['/home/nlourenco/open-analog-ics/BAIcO', '/home/nlourenco/open-analog-ics/BAIcO/src', '/home/nlourenco/open-analog-ics/BAIcO/test', '/home/nlourenco/open-analog-ics/BAIcO/notebooks', '/home/nlourenco/.vscode-server/extensions/ms-toolsai.jupyter-2021.6.999406279/pythonFiles', '/home/nlourenco/.vscode-server/extensions/ms-toolsai.jupyter-2021.6.999406279/pythonFiles/lib/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/nlourenco/open-analog-ics/BAIcO/myvenv/lib/python3.8/site-packages', '/home/nlourenco/open-analog-ics/BAIcO/myvenv/lib/python3.8/site-packages/IPython/extensions', '/home/nlourenco/.ipython']


In [ ]:
import os
os.environ['PYTHONPATH'].split(os.pathsep)

['./src/', './test/', './notebooks/', '']